Groundwater | Case Study

# Transport Modeling in Groundwater: From Verification to Application

Dr. Xiang-Zhao Kong & Dr. Beatrice Marti & Louise Noël du Payrat

## 1. Overview: Learning Path for Contaminant Transport Modeling

### 1.1 The Problem: Contaminant Plume Migration

Understanding how contaminants move through groundwater is critical for:
- **Risk assessment**: Will contamination reach drinking water wells?
- **Remediation design**: Where should we place extraction wells?
- **Regulatory compliance**: Meeting cleanup standards
- **Public health protection**: Protecting water supplies

This notebook teaches you how to model contaminant transport using MT3D-USGS, starting with verification and building to real-world application.

### 1.2 The Approach: Simple to Complex

We build understanding through a structured progression:

1. **Analytical solution** (Section 3)
   - 1D pulse source with known solution
   - Provides verification baseline
   - Understand fundamental transport behavior

2. **1D numerical verification** (Section 4)
   - Implement same problem in MT3D-USGS
   - Verify MT3D reproduces analytical solution
   - Build confidence in numerical code

3. **Load regional flow model** (Section 5)
   - Limmat Valley steady-state flow field
   - Foundation for transport modeling
   - Already calibrated from flow case study

4. **Method selection** (Section 6)
   - When to use different approaches
   - Trade-offs between methods
   - Professional practice guidance

5. **Telescope submodel with transport** (Section 7)
   - Create refined submodel around wells
   - Appropriate grid resolution for transport
   - Industry-standard workflow

6. **Your case study** (Section 8)
   - Apply telescope approach
   - Analyze contaminant transport
   - Well capture and protection zones

**The key insight:** We verify MT3D works (Section 4), then apply telescope refinement (Section 7) to get appropriate grid resolution. This is the workflow you'll use for your case studies.

### 1.3 Why This Structure?

**Building competence through verification:**
1. Start with known analytical solution
2. Verify numerical code matches it
3. Apply to complex real-world problem

**Practical workflow:**
1. Regional flow model (coarse grid)
2. Local refinement for transport (fine grid)
3. This is how professionals do it!

**Learning by doing:**
- Each section builds on previous
- Theory → verification → application
- Prepares you for independent case study work

### 1.4 Learning Outcomes

By the end of this notebook, you will be able to:

1. **Understand transport processes**: advection, dispersion, sorption, decay
2. **Verify numerical models** against analytical solutions
3. **Set up MT3D-USGS** for transport simulations
4. **Apply telescope refinement** for grid resolution challenges
5. **Select appropriate methods** for different problems
6. **Implement a complete transport case study** independently


### 1.5 Notebook Structure

This notebook guides you through transport modeling with increasing complexity:

**Section 1:** Overview and learning path

**Section 2:** Transport theory fundamentals (including grid resolution basics)

**Section 3:** Analytical solution for verification

**Section 4:** 1D MT3D verification (proves MT3D works correctly)

**Section 5:** Load Limmat Valley regional flow model

**Section 6:** Method selection framework (when to use different approaches)

**Section 7:** Telescope submodel implementation (refined grid + transport)

**Section 8:** Your transport case study workflow

**The key insight:** We verify MT3D works (Section 4), then apply telescope refinement with transport (Section 7) because we need fine resolution around wells. This is the approach you'll use for your case studies.

### 1.6 The Key Insight

**The telescope approach is essential for practical transport modeling.**

**Problem:** Transport needs fine grid resolution for accuracy  
**Challenge:** Fine grids everywhere = models too large  
**Solution:** Telescope refinement = fine grid only where needed

This is **industry-standard practice**:
- Regional flow model with coarse grid
- Refined submodel around sources/wells
- Transport on refined grid with appropriate resolution

**You will apply this workflow in your transport case study.**

---
## 2. Introduction to Contaminant Transport Modeling

Contaminant transport in groundwater is governed by physical, chemical, and biological processes. Understanding these processes is essential for predicting plume migration, assessing risks, and designing remediation strategies.

### 2.1 Fundamental Transport Processes

**Advection** is the movement of dissolved contaminants with flowing groundwater:

$$u = \frac{K \cdot i}{\phi_e}$$

where $u$ is pore water velocity (m/day), $K$ is hydraulic conductivity (m/day), $i$ is hydraulic gradient (m/m), and $\phi_e$ is effective porosity (dimensionless).

**Dispersion** causes spreading of the contaminant plume beyond what advection alone would predict. It results from:
- **Mechanical dispersion**: Variations in velocity at pore scale and between flow paths
- **Molecular diffusion**: Random motion of molecules (typically negligible compared to mechanical dispersion)

**Sorption** is the attachment of contaminants to aquifer solids, causing **retardation**:

$$R = 1 + \frac{(1-\phi_e) \cdot \rho_s \cdot K_s}{\phi_e}$$

where $R$ is the retardation factor (dimensionless), $\rho_s$ is solid density (kg/m³), and $K_s$ is the distribution coefficient (mL/g).

**Decay** represents biodegradation or chemical transformation:

$$c(t) = c_0 \cdot e^{-\lambda t}$$

where $\lambda$ is the first-order decay constant (1/day), related to half-life by $t_{1/2} = \ln(2)/\lambda$.

### 2.2 Governing Equation

The 3D advection-dispersion-reaction equation describes contaminant transport:

$$
R \frac{\partial c}{\partial t} = \nabla \cdot (D \nabla c) - \mathbf{u} \cdot \nabla c - \lambda R c
$$

where:
- $c$ = concentration (mg/L or kg/m³)
- $t$ = time (days)
- $D$ = dispersion tensor (m²/day)
- $\mathbf{u}$ = velocity vector (m/day)
- $\lambda$ = decay constant (1/day)

### 2.3 Formulation of the Transport Problem with Forcing Terms

#### Complete Transport Equation with Sources/Sinks

The **full form** of the transport equation includes **forcing terms** (sources and sinks):

$$
R \frac{\partial c}{\partial t} = \nabla \cdot (D \nabla c) - \nabla \cdot (\mathbf{u} c) - \lambda R c + \frac{q_s c_s}{\phi_e}
$$

where the **forcing term** $\frac{q_s c_s}{\phi_e}$ represents:
- $q_s$ = volumetric flow rate per unit volume of aquifer (1/day) - positive for injection, negative for extraction
- $c_s$ = concentration in the source/sink water (kg/m³)
- $\phi_e$ = effective porosity (dimensionless)

#### Physical Interpretation of Forcing Terms

**1. Point Sources (Constant Concentration)**
- Continuous spill or leak
- Implemented as: **Dirichlet boundary condition** $c(x_s, y_s, z_s, t) = c_0$
- MT3D implementation: `icbund = -1`, `sconc = c_0`

**2. Mass Loading Sources**
- Direct injection of mass into aquifer
- Rate: $\dot{M} = q_s \cdot c_s$ (kg/day)
- MT3D implementation: SSM package with `itype = 15`

**3. Well Sources/Sinks**
- **Injection well**: $q_s > 0$, introduces contaminant at concentration $c_s$
- **Pumping well**: $q_s < 0$, removes water at ambient concentration $c$
- MT3D implementation: SSM package with `itype = 2` (coupled to MODFLOW WEL package)

**4. Boundary Sources**
- Rivers: SSM with `itype = 4` (RIV package)
- Drains: SSM with `itype = 3` (DRN package)  
- General head boundaries: SSM with `itype = 5` (GHB package)

#### Mathematical Form for Different Source Types

**Constant concentration source (Dirichlet BC):**
$$c(\mathbf{x}_s, t) = c_0 \quad \text{for} \quad \mathbf{x} \in \Omega_s$$

**Mass injection source:**
$$\frac{\partial c}{\partial t} = \ldots + \frac{\dot{M}(t)}{\phi_e \cdot V_{cell}}$$

where $\dot{M}(t)$ is the mass injection rate (kg/day) and $V_{cell}$ is the cell volume (m³).

**Well extraction (sink):**
$$\frac{\partial c}{\partial t} = \ldots - \frac{Q_{well} \cdot c}{\phi_e \cdot V_{cell}}$$

where $Q_{well}$ is the pumping rate (m³/day), removing water at the local concentration $c$.

#### Units and Consistency

**CRITICAL:** MT3D-USGS requires consistent units with MODFLOW:

| Quantity | MODFLOW Unit | MT3D Unit | Conversion |
|----------|--------------|-----------|------------|
| Length | m | m | - |
| Time | days | days | - |
| Mass | - | kg | - |
| Concentration | - | kg/m³ | mg/L × 0.001 |
| Velocity | m/day | m/day | - |
| Dispersion coeff | m²/day | m²/day | - |
| Well flow | m³/day | m³/day | - |
| Mass rate | - | kg/day | - |

**Example conversions:**
- 100 mg/L = 0.1 kg/m³
- 1 g/L = 1 kg/m³
- Well at 1000 m³/day with 50 mg/L → injects 0.05 kg/day of mass

#### Boundary and Initial Conditions

**Initial condition (t = 0):**
$$c(\mathbf{x}, 0) = c_0(\mathbf{x})$$

Typically $c_0 = 0$ everywhere except at sources.

**Boundary conditions:**

1. **Dirichlet (specified concentration):**
   $$c = c_b \quad \text{on} \quad \Gamma_D$$
   
2. **Neumann (specified flux):**
   $$-D \nabla c \cdot \mathbf{n} = F_b \quad \text{on} \quad \Gamma_N$$

3. **Cauchy (flux proportional to concentration):**
   $$-D \nabla c \cdot \mathbf{n} + \mathbf{u} \cdot \mathbf{n} \, c = 0 \quad \text{on} \quad \Gamma_C$$

4. **No-flow boundary:**
   $$\nabla c \cdot \mathbf{n} = 0 \quad \text{on} \quad \Gamma_{NF}$$

where $\mathbf{n}$ is the outward normal vector on the boundary.

#### Solution Strategy in MT3D-USGS

MT3D solves the transport equation using:

1. **Operator splitting**:
   - Step 1: Advection (MOC, HMOC, MMOC, or TVD scheme)
   - Step 2: Dispersion/diffusion (implicit finite difference)
   - Step 3: Reactions (analytical or implicit)
   - Step 4: Sources/sinks (SSM package)

2. **Coupling with MODFLOW**:
   - Read velocity field from `.cbc` (cell-by-cell) file
   - Read well/boundary fluxes from MODFLOW output
   - Apply source concentrations via SSM package

3. **Mass balance**:
   MT3D tracks mass entering/leaving via:
   - Constant concentration boundaries
   - Wells (injection/extraction)
   - Boundaries (rivers, drains, GHB)
   - Decay/reactions

#### Key Takeaways for Modeling

✓ **Sources must be properly configured** - Wrong unit conversions → wrong results  
✓ **Well injection ≠ constant concentration** - Different mathematical formulations  
✓ **Mass balance is essential** - Always check mass conservation in `.mas` file  
✓ **Boundary conditions matter** - Plumes must stay within domain or have proper BCs  
✓ **Forcing terms are additive** - Multiple sources sum together in the equation

**Next section** covers analytical vs. numerical solutions, where we'll see how forcing terms appear in analytical solutions (e.g., the pulse solution uses an instantaneous point mass source at x=0).

### 2.4 Analytical vs. Numerical Solutions

**Analytical solutions** exist for simplified conditions (1D, uniform flow, simple boundaries). **Example**: solution for 1D pulse source:

$$c(x,t) = \frac{M/(\phi_e \cdot A)}{\sqrt{4 \pi D_L t}} \exp\left(-\frac{(x - ut)^2}{4 D_L t}\right) \exp(-\lambda t)$$

where:
- $M$ = mass released (kg)
- $\phi_e$ = effective porosity (dimensionless)
- $A$ = cross-sectional area (m²)
- $D_L$ = longitudinal dispersion coefficient (m²/day)
- $u$ = average linear velocity (m/day)   
- $ \lambda $ = decay constant (1/day)
- $x$ = distance from source (m)
- $t$ = time (days)
- $c(x,t)$ = concentration at location $x$ and time $t$ (kg/m³)

**Numerical solutions** (MT3D-USGS, MODFLOW 6 GWT) handle complex 2D/3D geometries, heterogeneity, wells, and boundaries.

### 2.5 MT3D-USGS: Modular Transport Simulator

MT3D-USGS (Bedekar et al., 2016) is the USGS version of MT3DMS, coupled with MODFLOW. It uses a **modular package structure**:

| Package | Purpose | Key Parameters |
|---------|---------|----------------|
| **BTN** | Basic Transport | Porosity, initial concentration, time steps |
| **ADV** | Advection | Numerical scheme (MOC, TVD, etc.) |
| **DSP** | Dispersion | Dispersivity ($\alpha_L$, $\alpha_T$, $\alpha_V$) |
| **SSM** | Source-Sink Mixing | Source locations and concentrations |
| **RCT** | Reactions | Sorption ($K_d$), decay ($\lambda$) |
| **GCG** | Solver | Convergence criteria |

MT3D reads flow velocities from MODFLOW output and solves transport on the same (or refined) grid.

### 2.6 When to Use Which Method?

```
Simple problem? → Try analytical first
├─ Uniform flow, 1D, no wells → Ogata-Banks
├─ Conservative tracer only → Particle tracking (MODPATH)
└─ Need verification → Always compare numerical to analytical

Complex problem? → Use numerical model
├─ Wells, boundaries, heterogeneity → MT3D-USGS
├─ Multiple contaminants → MT3D-USGS (multi-species)
└─ Regulatory/design decisions → MT3D-USGS (defensible)
```

**Recommendation**: Start simple, add complexity only when justified. Always verify numerical models against analytical solutions.

### 2.7 A Note on Grid Resolution and the numerical Peclet Number
When using numerical methods like MT3D, grid resolution is critical for accurate transport modeling. The **numerical Peclet number (Pe)** is a dimensionless number that helps us assess whether our grid is fine enough:

$$Pe = \frac{\Delta x}{\alpha_L}$$

Where:

- $\Delta x$ is the grid cell size [L]
- $\alpha_L$ is the longitudinal dispersivity [L]
 
**Rule of thumb**: Keep Pe ≤ 2 (ideally ≤ 1) to avoid numerical dispersion artifacts. This means your grid cells should be smaller than or equal to 2 times the dispersivity.

For example:

- If $\alpha_L = 20$ m, then $\Delta x \leq 40$ m (better if $\Delta x \leq 20$ m)
- Smaller dispersivity requires finer grids
- Around wells or sources, local grid refinement is often needed
 
This is why we'll use the **telescope approach** (local grid refinement) for our case study—it gives us fine resolution where we need it without making the entire model too large.

### 2.8 References

- Sale, T., & Scalia, J. (2025). Modern Subsurface Contaminant Hydrology, first edition. The Groundwater Project. [https://doi.org/10.62592/IQDO4854](https://doi.org/10.62592/IQDO4854).
- Bedekar, Vivek, Morway, E.D., Langevin, C.D., and Tonkin, Matt, 2016, MT3D-USGS version 1: A U.S. Geological Survey release of MT3DMS updated with new and expanded transport capabilities for use with MODFLOW: U.S. Geological Survey Techniques and Methods 6-A53, 69 p., [https://dx.doi.org/10.3133/tm6A53](https://dx.doi.org/10.3133/tm6A53).
- Ogata, A., & Banks, R.B. (1961). *Longitudinal Dispersion in Porous Media*. USGS Professional Paper 411-A. [https://pubs.usgs.gov/pp/0411a/report.pdf](https://pubs.usgs.gov/pp/0411a/report.pdf).
- Gelhar, L.W., et al. (1992). Field-scale dispersion in aquifers. *Water Resources Research*, 28(7), 1955-1974. [https://doi.org/10.1029/92WR00607](https://doi.org/10.1029/92WR00607).

---

Let's now see these concepts in practice with code examples in Section 3.

---
## 3. Analytical Solution: Pulse Source for 1D Transport

Before building any numerical models, let's first understand the analytical solution. Though you have covered other solutions in class, we will focus on the pulse source solution here for simplicity reasons. The pulse source solution provides the "correct" answer for 1D transport with uniform flow and an instantaneous concentration source.

You can choose to use other solutions, already implemented for example in the adepy package, for your case study projects. 

### 3.1 The Pulse Source Solution

For 1D transport along a semi-infinite domain with:
- Uniform groundwater velocity $u$ (m/day)
- Longitudinal dispersion coefficient $D_L = \alpha_L \cdot u$ (m²/day)
- Instantaneous source release of Mass $M$ at $x = 0$
- Initially clean aquifer ($C = 0$ at $t = 0$)

The concentration at distance $x$ and time $t$ is:

$$c(x,t) = \frac{M/(\phi_e \cdot A)}{\sqrt{4 \cdot \pi \cdot D_L \cdot t}} \cdot \exp{\left[\frac{-(x - u \cdot t)^2}{4 \cdot D_L \cdot t}\right]}$$

assuming that the mass M is released instantaneously at t=0 at location x=0.

**Physical interpretation:**
- Advective-dispersive front moving at velocity $u$
- **At source ($x=0$)**: $C(0,t) = C_0$ (matches boundary condition)

### 3.2 Implement Pulse Source Function

Let's look at the analytical solution as implemented in the adepy package and compare the function to the analytical expression above.

In [ ]:
from scipy.special import erfc
import numpy as np
import matplotlib.pyplot as plt
! pip install adepy
import adepy
from adepy.uniform.oneD import pulse1

# Implementation of the analytical solution to 1D transport from an instantaneous 
# pulse source as implemented in adepy. In order not to overwrite the adepy function,
# we name this function pulse1_demo. We import the original adepy function 
# in the line above.
def pulse1_demo(m0, x, t, v, n, al, xc=0.0, Dm=0.0, lamb=0.0, R=1.0):
    """Compute the 1D concentration field of a dissolved solute from an instantaneous pulse point source in an infinite aquifer
    with uniform background flow.

    Source: [bear_1979]

    The one-dimensional advection-dispersion equation is solved for concentration at specified `x` location(s) and
    output time(s) `t`. An infinite system with uniform background flow in the x-direction is subjected to a pulse source
    with mass `m0` at `xc` at time `t=0`.
    The solute can be subjected to 1st-order decay. Since the equation is linear, multiple sources can be superimposed
    in time and space.
    Note that the equation has the same shape as the probability density function of a Gaussian distribution.

    The mass center of the plume at a given time `t` can be found at `x=xc + v*t/R`.

    Parameters
    ----------
    m0 : float
        Source mass [M].
    x : float or 1D of floats
        x-location(s) to compute output at [L].
    t : float or 1D of floats
        Time(s) to compute output at [T].
    v : float
        Average linear groundwater flow velocity of the uniform background flow in the x-direction [L/T].
    n : float
        Aquifer porosity. Should be between 0 and 1 [-].
    al : float
        Longitudinal dispersivity [L].
    xc : float
        x-coordinate of the point source [L], defaults to 0.0.
    Dm : float, optional
        Effective molecular diffusion coefficient [L**2/T]; defaults to 0 (no molecular diffusion).
    lamb : float, optional
        First-order decay rate [1/T], defaults to 0 (no decay).
    R : float, optional
        Retardation coefficient [-]; defaults to 1 (no retardation).

    Returns
    -------
    ndarray
        Numpy array with computed concentrations [M/L**3] at location(s) `x` and time(s) `t`.

    References
    ----------
    .. [bear_1979] Bear, J., 1979. Hydraulics of Groundwater. New York, McGraw Hill, 596 p.

    """
    x = np.atleast_1d(x)
    t = np.atleast_1d(t)

    D = al * v + Dm

    # apply retardation coefficient to right-hand side
    v = v / R
    D = D / R

    term0 = (
        1
        / (n * np.sqrt(4 * np.pi * D * t))
        * np.exp(-((x - xc - v * t) ** 2) / (4 * D * t) - lamb * t)
    )

    return m0 * term0

# Test the function with example parameters
x_test = np.array([0, 50, 100, 150, 200, 250, 300])  # m
t_test = 365  # 1 year
M_test = 10  # kg
u_test = 0.4  # m/day
alpha_L_test = 10.0  # m
D_test = alpha_L_test * u_test  # 4.0 m²/day
phi_e_test = 0.2  # effective porosity

C_test = pulse1(
    m0=M_test, 
    x=x_test,
    t=t_test,
    v=u_test,
    n=phi_e_test, 
    al=alpha_L_test)

print("Pulse source function test:")
print(f"Parameters:")
print(f"  Source mass: {M_test} kg ")
print(f"  Velocity: {u_test} m/day")
print(f"  Dispersivity: {alpha_L_test} m")
print(f"  Dispersion coefficient: {D_test} m²/day")
print(f"  Time: {t_test} days ({t_test/365:.1f} years)")
print(f"\nConcentrations at different distances:")
for i, x_val in enumerate(x_test):
    print(f"  x = {x_val:3.0f} m: C = {C_test[i]:.6f} kg/m³ ({C_test[i]*1e3:.1f} mg/L)")


The output concentrations at different distances after 1 year of transport from an instantaneous pulse source find a maximum around the distance of 150 m.  

> **At which distance is the peak concentration located after 1 year?**
<details>
<summary>Solution</summary>
You can calculate the distance of peak = velocity * time: 0.4 m/day * 365 days = 146 m
</details>

### 3.3 Visualize Analytical Solution
Let's visualize the analytical solution of the 1D instantaneous pulse source problem for different times and distances.

In [ ]:
# Parameter sensitivity analysis
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

t_sensitivity = 365  # Fixed time (1 year)
x_sens = np.linspace(0, 400)

# Left plot: Effect of velocity
ax1 = axes[0]
velocities = [0.2, 0.4, 0.8]  # m/day
alpha_L_fixed = 10.0  # m

for u_sens in velocities:
    D_sens = alpha_L_fixed * u_sens
    C_sens = pulse1(m0=M_test, x=x_sens, t=t_sensitivity, v=u_sens, n=phi_e_test, al=alpha_L_fixed)
    ax1.plot(x_sens, C_sens, linewidth=2, 
             label=f'u = {u_sens} m/day (front at {u_sens*t_sensitivity:.0f}m)')

ax1.set_xlabel('Distance from Source (m)', fontsize=11)
ax1.set_ylabel('Concentration (kg/m3)', fontsize=11)
ax1.set_title(f'Effect of Velocity (t = {t_sensitivity} days, αL = {alpha_L_fixed} m)', fontsize=12)
ax1.grid(alpha=0.3)
ax1.legend(fontsize=9)
ax1.set_xlim(0, 400)

# Right plot: Effect of dispersivity
ax2 = axes[1]
u_fixed = 0.4  # m/day
dispersivities = [5.0, 10.0, 20.0]  # m

for alpha_L_sens in dispersivities:
    D_sens = alpha_L_sens * u_fixed
    C_sens = pulse1(m0=M_test, x=x_sens, t=t_sensitivity, v=u_fixed, n=phi_e_test, al=alpha_L_sens)
    ax2.plot(x_sens, C_sens, linewidth=2, 
             label=f'αL = {alpha_L_sens} m (D = {D_sens:.1f} m²/day)')

ax2.set_xlabel('Distance from Source (m)', fontsize=11)
ax2.set_ylabel('Concentration (kg/m3)', fontsize=11)
ax2.set_title(f'Effect of Dispersivity (t = {t_sensitivity} days, v = {u_fixed} m/day)', fontsize=12)
ax2.grid(alpha=0.3)
ax2.legend(fontsize=9)
ax2.set_xlim(0, 400)

plt.tight_layout()
plt.show()

print("\nParameter sensitivity insights:")
print("  • Velocity controls front position (higher v → faster migration)")
print("  • Dispersivity controls spreading (higher αL → more dispersion)")
print("  • Both affect plume shape and maximum concentration")

### 3.4 Summary: Analytical Solution Established

**What we accomplished in Section 3:**

✅ **Implemented 1D instantaneous pulse solution** in Python  
✅ **Visualized plume evolution** over time  
✅ **Explored parameter sensitivity** (velocity and dispersivity)

**Key learning outcomes:**

1. **Understand the physics** - Analytical solution shows pure advection-dispersion
2. **Know the "correct" answer** - This is our benchmark for verification
3. **Parameter intuition** - Velocity moves plume, dispersivity spreads it
4. **Baseline established** - Now we can test if MT3D reproduces this

**Next:** Section 4 will implement a 1D numerical model with MT3D-USGS and compare results to this analytical solution. We expect excellent agreement (< 10% error) when grid resolution is adequate (Pe ≤ 2).

---
## 4. Numerical Verification: 1D MT3D-USGS Model

Now that we've established the analytical solution in Section 3, let's verify that MT3D-USGS can reproduce it. This builds confidence that the numerical code works correctly before applying it to complex problems.

### 4.1 Why Verify Against Analytical Solution?

**Professional practice:** Always test numerical models on simple cases with known solutions before tackling complex problems.

**Benefits:**
1. **Build confidence** - Verify MT3D physics are correct
2. **Understand parameters** - Learn how grid resolution affects results
3. **Test workflow** - Debug setup issues in simple context
4. **Establish baseline** - Know what "good agreement" looks like
5. **Learn resolution requirements** - See effect of grid size and time step

**The plan:**
- Set up simple 1D column with uniform flow
- Use **fine grid** (Δx = 5m) → Pe = 5/10 = **0.5** ✓
- Use small time steps (Δt = 2.5 days) → Cr = **0.8** ✓
- Compare MT3D results to the 1D analytical solution from Section 3
- Expect excellent agreement (within 5-10%)

### 4.2 Model Design for Verification

**Key parameters matching Section 3 analytical solution:**
- Domain: 500m long, 50m wide, 10m thick (1 layer)
- Grid: Δx = 5m (100 cells) → **Pe = 5/10 = 0.5** ✓
- Hydraulic gradient: i = 0.002 (uniform)
- Hydraulic conductivity: K = 50 m/day
- Porosity: n = 0.25
- Seepage velocity: v = K·i/n = 50 × 0.002 / 0.25 = **0.4 m/day**
- Dispersivity: αL = 10 m
- Dispersion coefficient: D = 10 × 0.4 = **4 m²/day**
- Time step: Δt = 2.5 days → **Cr = 0.4·2.5/(0.25·5) = 0.8** ✓
- Simulation: 1000 days

**Note:** These match the parameters we used in Section 3.2 for testing the analytical solution!

### 4.3 Set Up 1D MODFLOW Model

First, we need to create a MODFLOW model that produces uniform flow.

In [ ]:
# Import additional libraries for 1D verification model
import flopy

# Define workspace
import pathlib
from pathlib import Path

transport_ws = Path.home() / "applied_groundwater_modelling_data" / "limmat" / "transport"
transport_ws.mkdir(exist_ok=True)

# Create workspace for 1D model
model_1d_ws = transport_ws / "test_1d"
model_1d_ws.mkdir(exist_ok=True)

# Model parameters for 1D uniform flow
Lx = 500.0   # Length (m)
Ly = 1.0    # Width (m) - arbitrary for 1D
Lz = 1.0    # Thickness (m)

nlay = 1
nrow = 1  # 1D model: single row
ncol = 500  # 500 cells in x-direction

delr = np.ones(ncol) * (Lx / ncol)  # Δx = 1m
delc = np.ones(nrow) * Ly
top = 1.0
botm = np.array([0.0])

# Hydraulic properties
hk = 50.0  # Hydraulic conductivity (m/day)
sy = 0.25  # Specific yield (= porosity phi_e for unconfined)
ss = 1e-5  # Specific storage (1/m)

# Target gradient and velocity
target_gradient = 0.002  # i = 0.002
target_velocity = hk * target_gradient / sy  # u = K·i/phi_e = 0.4 m/day

print(f"1D Model Setup:")
print(f"  Domain: {Lx}m × {Ly}m × {Lz}m")
print(f"  Grid: {ncol} cells × {nrow} row × {nlay} layer")
print(f"  Cell size: Δx = {delr[0]:.1f}m, Δy = {delc[0]:.1f}m")
print(f"  Hydraulic conductivity: K = {hk} m/day")
print(f"  Porosity: phi_e = {sy}")
print(f"  Target gradient: i = {target_gradient}")
print(f"  Expected velocity: u = {target_velocity:.2f} m/day")

# Create MODFLOW model
modelname_1d = 'test_1d'
mf_1d = flopy.modflow.Modflow(modelname_1d, model_ws=str(model_1d_ws), exe_name='mfnwt', version='mfnwt')

# DIS package
dis_1d = flopy.modflow.ModflowDis(
    mf_1d, nlay=nlay, nrow=nrow, ncol=ncol,
    delr=delr, delc=delc,
    top=top, botm=botm,
    nper=1, perlen=1, nstp=1, steady=True
)

# BAS package - all cells active
ibound = np.ones((nlay, nrow, ncol), dtype=np.int32)
strt = top  # Initial head

bas_1d = flopy.modflow.ModflowBas(mf_1d, ibound=ibound, strt=strt)

# UPW package (Upstream Weighting, for MODFLOW-NWT)
upw_1d = flopy.modflow.ModflowUpw(
    mf_1d, 
    hk=hk,      # Horizontal hydraulic conductivity
    vka=hk,     # Vertical hydraulic conductivity (same as horizontal)
    sy=sy,      # Specific yield
    ss=ss,      # Specific storage
    laytyp=1,   # Layer type (1 = convertible/unconfined)
    iphdry=0,    # Flag for dry cell handling
    ipakcb=53   # Budget output unit number
)
# CHD package - constant heads at boundaries to create uniform gradient
# Left boundary (col=0): head = 10m
# Right boundary (col=99): head = 10 - Lx*i = 10 - 500*0.002 = 9m
stress_period_data = {}
chd_list = []

# Left boundary (high head)
for lay in range(nlay):
    for r in range(nrow):
        chd_list.append([lay, r, 0, 10.0, 10.0])  # [lay, row, col, shead, ehead]

# Right boundary (low head)
head_right = 10.0 - Lx * target_gradient
for lay in range(nlay):
    for r in range(nrow):
        chd_list.append([lay, r, ncol-1, head_right, head_right])

stress_period_data[0] = chd_list
chd_1d = flopy.modflow.ModflowChd(
    mf_1d, 
    stress_period_data=stress_period_data,
    ipakcb=53  # Budget output unit number
)

# OC package
oc_1d = flopy.modflow.ModflowOc(
    mf_1d, 
    stress_period_data={(0,0): ['save head', 'save budget']},
    budget_unit=53  # Explicitly set budget unit number, or budget output may not be written
)

# NWT solver
nwt_1d = flopy.modflow.ModflowNwt(mf_1d)

# LMT package for MT3D coupling
lmt_1d = flopy.modflow.ModflowLmt(mf_1d, output_file_name='mt3d_link.ftl')

# Write and run
mf_1d.write_input()
print("\n✓ 1D MODFLOW model created")
print(f"✓ Model files written to: {model_1d_ws}")

# Check model
mf_1d.check()

In [ ]:
# Run 1D MODFLOW
print("Running 1D MODFLOW...")
success_1d, buff_1d = mf_1d.run_model(silent=True)

if success_1d:
    print("✓ 1D MODFLOW converged successfully")
    
    # Load head file
    hds_1d = flopy.utils.HeadFile(str(model_1d_ws / f"{modelname_1d}.hds"))
    heads_1d = hds_1d.get_data()
    
    # Load cell budget file for flow data
    cbc_1d = flopy.utils.CellBudgetFile(str(model_1d_ws / f"{modelname_1d}.cbc"))
    
    # Extract flow-right-face (FRF) or flow data
    # Options: 'FLOW RIGHT FACE', 'FLOW FRONT FACE', 'FLOW LOWER FACE'
    frf = cbc_1d.get_data(text='FLOW RIGHT FACE')[0]  # [0] gets first (and only) stress period
    
    # Calculate velocity from specific discharge
    # frf has shape (nlay, nrow, ncol)
    # Specific discharge q = Q / (width * height)
    # Velocity v = q / porosity (or effective porosity/sy for transport)
    
    # For 1D case: extract flow at middle of domain
    q_x = frf[0, 0, ncol//2]  # Specific discharge at mid-domain [m³/day/m²] or [m/day]
    velocity_actual = abs(q_x) / sy  # Divide by porosity to get pore velocity
    
    # Calculate gradient from heads
    head_left = heads_1d[0, 0, 0]
    head_right = heads_1d[0, 0, -1]
    gradient_actual = (head_left - head_right) / Lx
    
    print(f"\nFlow verification:")
    print(f"  Head at left (col 0): {head_left:.4f} m")
    print(f"  Head at right (col 99): {head_right:.4f} m")
    print(f"  Gradient: i = {gradient_actual:.6f} (target: {target_gradient})")
    print(f"  Specific discharge: q = {abs(q_x):.4f} m/day")
    print(f"  Pore velocity: v = {velocity_actual:.4f} m/day (target: {target_velocity:.2f})")
    
    if abs(velocity_actual - target_velocity) < 0.01:
        print("  ✓ Uniform flow achieved!")
    else:
        print("  ⚠ Warning: Flow not perfectly uniform")
else:
    print("✗ 1D MODFLOW failed")
    print("buff_1d:", buff_1d)

### 4.4 Set Up 1D MT3D Model

Now create the MT3D model with proper resolution (Pe ≤ 2) to match our analytical parameters.

**Important Note on Instantaneous Mass Release Implementation:**

For an **instantaneous pulse source**, we use the **initial concentration** (`sconc` in BTN package):

- **At t=0**: Mass M is placed in source cell → `c(0,0,0,t=0) = M / (V_cell × φ)`
- **For t>0**: Mass advects and disperses away (source cell concentration **decreases**)
- **Key**: `icbund = 1` (active cell) allows concentration to change, NOT `icbund = -1` (constant concentration)

This is different from a **continuous source**, which would require:
- `icbund = -1` to maintain constant concentration, OR
- SSM package with mass injection rate at every time step

Our implementation correctly represents a **one-time instantaneous release** at t=0.

In [ ]:
# Transport parameters
porosity_1d = 0.25
M_instantaneous = 10  # kg 
alpha_L_1d = 10.0  # Longitudinal dispersivity (m)

# Check the numerical Peclet number
Pe_1d = delr[0] / alpha_L_1d
print(f"Grid resolution check:")
print(f"  Δx = {delr[0]:.1f} m")
print(f"  αL = {alpha_L_1d} m")
print(f"  Pe = Δx/αL = {Pe_1d:.2f}")
print(f"  Δx is the cell length in x-direction")
if Pe_1d <= 2:
    print(f"  ✓ Pe ≤ 2: Good resolution for transport!")
else:
    print(f"  ❌ Pe > 2: Grid too coarse!")

# Time stepping with Courant check
# Cr = v·Δt/(n·Δx) ≤ 1
# Get velocity from previous MODFLOW run, or calculate if not available
if 'velocity_actual' in locals():
    u_1d = velocity_actual
else:
    # Calculate expected velocity if MODFLOW not run yet
    target_gradient = 0.002
    hk = 50.0
    u_1d = hk * target_gradient / porosity_1d  # u = K·i/n
    print(f"\nUsing calculated velocity: {u_1d:.2f} m/day")
    print(f"(Run Section 4.4 first to use actual MODFLOW velocity)")

# Simulation time
sim_time_1d = 1000  # days

dx_1d = delr[0]
dt_max = porosity_1d * dx_1d / u_1d  # Maximum stable time step
dt_1d = 1.0  # days - Increased to reduce number of steps while maintaining stability
if dt_1d > dt_max:
    print(f"⚠ Chosen Δt = {dt_1d} days exceeds max stable Δt = {dt_max:.2f} days")
    print("  Adjusting Δt to max stable value.")
    dt_1d = dt_max
    
# CRITICAL: For instantaneous mass release, we use initial concentration
# Calculate equivalent initial concentration in source cell
cell_volume = delr[0] * delc[0] * Lz  # m³
pore_volume = cell_volume * porosity_1d  # m³ of pore space
C_initial = M_instantaneous / pore_volume  # kg/m³

# MT3D has a maximum limit on time steps per stress period
# Many versions have MXSTP = 1000, so we need to stay well below this
max_steps_per_period = 1000  # MT3D MXSTP limit (conservative)
total_steps = int(np.ceil(sim_time_1d / dt_1d))

if total_steps > max_steps_per_period:
    # Adjust dt to stay within limit
    dt_1d = np.ceil(sim_time_1d / max_steps_per_period * 10) / 10  # Round to 0.1
    total_steps = int(np.ceil(sim_time_1d / dt_1d))
    print(f"⚠ Adjusting Δt to {dt_1d:.2f} days to stay within MT3D step limit ({max_steps_per_period})")

# Use single stress period 
nper_1d = 1
perlen_1d = [sim_time_1d]
nstp_1d = total_steps
Cr_1d = u_1d * dt_1d / (porosity_1d * dx_1d)

print(f"\nInstantaneous mass release setup:")
print(f"  Total mass: {M_instantaneous} kg")
print(f"  Cell volume: {cell_volume:.2f} m³")
print(f"  Pore volume: {pore_volume:.2f} m³")
print(f"  Initial concentration at source: {C_initial:.6f} kg/m³")

print(f"\nTime stepping:")
print(f"  Total simulation time: {sim_time_1d} days")
print(f"  Δt = {dt_1d:.2f} days")
print(f"  Total steps: {nstp_1d}")
print(f"  Courant number: Cr = {Cr_1d:.2f}")
if Cr_1d <= 1.0:
    print(f"  ✓ Cr ≤ 1: Numerically stable!")
else:
    print(f"  ⚠ Cr > 1: May have stability issues")

# Create MT3D model
mt_1d = flopy.mt3d.Mt3dms(
    modelname='mt_' + modelname_1d,
    model_ws=str(model_1d_ws),
    exe_name='mt3dms',
    modflowmodel=mf_1d
)

# BTN package
# All cells are active (no constant concentration boundaries)
icbund_1d = np.ones((nlay, nrow, ncol), dtype=np.int32)

# Initial concentration: instantaneous mass in source cell
sconc_1d = np.zeros((nlay, nrow, ncol), dtype=np.float32)

# We place the source location 20 meters into the first cell to avoid edge effects
source_offset = 20  # meters
# Because our cell size in x-direction is 1m, the index of the offset is 20 cells
sconc_1d[0, 0, 20] = C_initial  # kg/m³

prsity_1d = np.ones((nlay, nrow, ncol), dtype=np.float32) * porosity_1d

# Output times
nprs_1d = 20
timprs_1d = np.linspace(sim_time_1d / nprs_1d, sim_time_1d, nprs_1d)

btn_1d = flopy.mt3d.Mt3dBtn(
    mt_1d,
    ncomp=1, mcomp=1,
    prsity=prsity_1d,
    icbund=icbund_1d,
    sconc=sconc_1d,
    nper=nper_1d,
    perlen=perlen_1d,
    nstp=nstp_1d,
    timprs=timprs_1d
)

# ADV package - TVD scheme
adv_1d = flopy.mt3d.Mt3dAdv(mt_1d, mixelm=-1)

# DSP package
dsp_1d = flopy.mt3d.Mt3dDsp(
    mt_1d,
    al=alpha_L_1d, 
    trpt=0.1,
    trpv=0.01,
    dmcoef=0.0
)

# SSM package - no external sources (mass is in initial condition)
ssm_1d = flopy.mt3d.Mt3dSsm(mt_1d)

# GCG solver
gcg_1d = flopy.mt3d.Mt3dGcg(mt_1d, mxiter=100, iter1=50, isolve=3, cclose=1e-6)

# Write input
mt_1d.write_input()

print("\n✓ 1D MT3D model created")
print(f"  Pe = {Pe_1d:.2f} (excellent!)")
print(f"  Cr = {Cr_1d:.2f} (stable!)")
print(f"  Instantaneous mass release via initial concentration")

# Check model 
mt_1d.check()

### 4.5 Run 1D MT3D and Extract Results


In [ ]:
# Run MT3D
print("Running 1D MT3D transport simulation...")
success_mt_1d, buff_mt_1d = mt_1d.run_model(silent=True)

# Check for output
ucn_file_1d = model_1d_ws / "MT3D001.UCN"

if ucn_file_1d.exists() and ucn_file_1d.stat().st_size > 0:
    print("✓ 1D MT3D completed successfully")
    
    # Load results
    ucn_1d = flopy.utils.UcnFile(str(ucn_file_1d))
    times_1d = ucn_1d.get_times()
    
    # Convert to numpy array if it's a list
    if isinstance(times_1d, list):
        times_1d = np.array(times_1d)
    
    print(f"  Output times: {len(times_1d)} snapshots")
    print(f"  Simulation time: 0 to {times_1d[-1]:.0f} days")
    
    # Extract concentration profiles at selected times
    times_to_plot = [50, 100, 250, 500]  # days
    conc_profiles_mt3d = {}
    
    for t in times_to_plot:
        # Find closest output time
        idx = np.argmin(np.abs(times_1d - t))
        t_actual = times_1d[idx]
        conc = ucn_1d.get_data(totim=t_actual)
        conc_profiles_mt3d[t_actual] = conc[0, 0, :]  # Extract 1D profile
    
    print(f"\n✓ Extracted {len(conc_profiles_mt3d)} concentration profiles for comparison")
    
else:
    print("✗ 1D MT3D failed - no concentration output")
    print("Check error messages above for details")
    conc_profiles_mt3d = None

# Velocity comparison for 1D transport model
print("\n" + "="*70)
print("VELOCITY VERIFICATION FOR 1D TRANSPORT MODEL")
print("="*70)

# Print the three velocity values
print(f"\nVelocity calculations (from Darcy's law: u = K·i/n):")
print(f"  1. Target velocity (expected):  u = {target_velocity:.4f} m/day")
if 'velocity_actual' in locals():
    print(f"  2. Actual velocity (MODFLOW):   u = {velocity_actual:.4f} m/day")
else:
    print(f"  2. Actual velocity (MODFLOW):   Not available (run Section 4.4 first)")

print(f"  3. Transport velocity (u_1d):   u = {u_1d:.4f} m/day")

# Compare velocities
print(f"\nVelocity used in analytical solution:")
print(f"  pulse1(v=u_1d) = {u_1d:.4f} m/day")

print(f"\nVelocity used in MT3D numerical model:")
print(f"  Calculated from MODFLOW flow field with porosity = {porosity_1d}")
print(f"  MT3D uses specific discharge (q) from MODFLOW divided by porosity")
print(f"  Effective velocity = q/n = {u_1d:.4f} m/day")

# Check if velocities match
print(f"\n{'='*70}")
if 'velocity_actual' in locals():
    velocity_diff = abs(velocity_actual - target_velocity)
    if velocity_diff < 0.01:
        print(f"✓ VERIFIED: Actual velocity matches target velocity")
        print(f"  |u_actual - u_target| = {velocity_diff:.6f} m/day < 0.01 m/day")
    else:
        print(f"✗ WARNING: Velocity mismatch!")
        print(f"  |u_actual - u_target| = {velocity_diff:.6f} m/day")

print(f"\n✓ CONFIRMED: Same velocity (u = {u_1d:.4f} m/day) used in both:")
print(f"  - Analytical solution (pulse1 function, parameter v)")
print(f"  - Numerical solution (MT3D coupled to MODFLOW)")
print(f"{'='*70}\n")

In [ ]:
# Verify instantaneous pulse behavior
# Check that source cell concentration DECREASES over time (not constant)
if conc_profiles_mt3d is not None:
    source_conc_over_time = []
    for t_actual in sorted(conc_profiles_mt3d.keys()):
        C_source = conc_profiles_mt3d[t_actual][0]  # Concentration in first cell
        source_conc_over_time.append((t_actual, C_source))
        
    # Calculate total mass in system
    print(f"\n=== Mass Conservation Check ===")
    cell_pore_volume = delr[0] * delc[0] * Lz * porosity_1d  # m³
    
    for t_actual in [sorted(conc_profiles_mt3d.keys())[0], sorted(conc_profiles_mt3d.keys())[-1]]:
        conc_profile = conc_profiles_mt3d[t_actual]
        total_mass = np.sum(conc_profile) * cell_pore_volume  # kg
        print(f"  t = {t_actual:6.1f} days: Total mass = {total_mass:.6f} kg (should be {M_instantaneous:.6f} kg)")
        error_pct = abs(total_mass - M_instantaneous) / M_instantaneous * 100
        print(f"                    Mass error = {error_pct:.2f}%")


The total solute mass is not perfectly conserved. This is an indication of numerical dispersion.  

### 4.6 Compare MT3D Results to Analytical Solution

Now let's compare the numerical (MT3D) and analytical solutions (from Section 3).

In [ ]:
if conc_profiles_mt3d is not None:
    # Create distance array (cell centers)
    x_centers = np.cumsum(delr) - delr/2
    
    # IMPORTANT: Set analytical source location to match MT3D source cell center
    xc_source = x_centers[0] + 20  # center of first cell + offset of 20m to avoid edge effects 
    
    # Calculate analytical solutions
    D_1d = alpha_L_1d * u_1d  # Dispersion coefficient
    
    # Create plots
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    axes = axes.flatten()
    
    # Calculate RMSE for each time
    rmse_values = []
    
    for idx, (t_actual, conc_mt3d) in enumerate(conc_profiles_mt3d.items()):
        ax = axes[idx]
        
        # Analytical solution WITH CORRECTED SOURCE LOCATION
        C_analytical = pulse1(m0=M_instantaneous, x=x_centers, t=t_actual, 
                              v=u_1d, n=porosity_1d, al=alpha_L_1d, xc=xc_source)
        
        # Convert to mg/L for plotting
        C_analytical_mg_L = C_analytical 
        C_mt3d_mg_L = conc_mt3d 
        
        # Plot both solutions
        ax.plot(x_centers, C_analytical_mg_L, 'b-', linewidth=2, 
                label='1D pulse (Analytical)', zorder=2)
        ax.plot(x_centers[::3], C_mt3d_mg_L[::3], 'ro', markersize=4, alpha=0.6, 
                label=f'MT3D (Numerical)', zorder=3)
        
        # Add plume front (from source location)
        x_front = xc_source + u_1d * t_actual
        ax.axvline(x_front, color='gray', linestyle='--', alpha=0.5, 
                  label=f'Advective front ({x_front:.0f}m)', zorder=1)
        
        # Mark source location
        ax.axvline(xc_source, color='red', linestyle=':', alpha=0.3, 
                  label=f'Source (xc={xc_source:.1f}m)', zorder=1)
        
        # Calculate RMSE (excluding source cell)
        rmse = np.sqrt(np.mean((C_analytical[1:] - conc_mt3d[1:])**2)) 
        rmse_values.append(rmse)
        
        # Calculate relative error
        max_C = M_instantaneous / (porosity_1d * np.sqrt(4 * np.pi * D_1d * t_actual))
        rel_error = (rmse / max_C) * 100
        
        ax.set_xlabel('Distance from Origin (m)', fontsize=11)
        ax.set_ylabel('Concentration (kg/m³)', fontsize=11)
        ax.set_title(f'Time = {t_actual:.0f} days ({t_actual/365:.2f} years)\n' + 
                    f'RMSE = {rmse:.3f} kg/m³ ({rel_error:.1f}%)', fontsize=11)
        ax.set_xlim(0, 400)
        ax.set_ylim(0, 1.1)
        ax.grid(alpha=0.3)
        ax.legend(fontsize=9, loc='upper right')
    
    plt.tight_layout()
    plt.show()
    
    # Summary statistics
    print("\n" + "="*70)
    print(f"COMPARISON (with xc = {xc_source:.1f} m)")
    print("="*70)
    print(f"\nAgreement metrics:")
    avg_rmse = np.mean(rmse_values)
    avg_rel_error = (avg_rmse / (max_C)) * 100
    print(f"  Average RMSE: {avg_rmse:.3f} kg/m³")
    print(f"  Average relative error: {avg_rel_error:.1f}%")
    
    if avg_rel_error < 10:
        print(f"\n  ✓ EXCELLENT agreement (< 10% error)")
        print(f"  ✓ MT3D-USGS accurately solves transport physics!")
        print(f"  ✓ Proper grid resolution (Pe = {Pe_1d:.1f}) is key!")
    elif avg_rel_error < 20:
        print(f"\n  ✓ GOOD agreement (< 20% error)")
        print(f"  ✓ Results are acceptable for most applications")
    else:
        print(f"\n  ⚠ POOR agreement (> 20% error)")
        print(f"  ⚠ Grid may need further refinement")
    
    print(f"\n{'='*70}")
    print("KEY TAKEAWAY:")
    print(f"By setting xc={xc_source:.1f}m to match the MT3D source cell center,")
    print("the analytical solution now properly aligns with the numerical model.")
    print("This eliminates the apparent velocity/position discrepancy!")
    print("="*70)
    
else:
    print("Skipping comparison - MT3D simulation failed")

### 4.7 Mass Conservation Comparison: Analytical vs. Numerical

One of the most important checks for transport models is **mass conservation**. For an instantaneous pulse source, the total mass in the system should remain constant over time (no sources, sinks, or losses at boundaries).

We'll compare:
1. **MT3D numerical model**: Integrate concentration over all cells
2. **Analytical solution**: Integrate the analytical concentration profile

Both should equal the initial mass (10 kg) at all times.

In [ ]:
if conc_profiles_mt3d is not None:
    print("\n" + "="*70)
    print("MASS CONSERVATION: ANALYTICAL vs. NUMERICAL COMPARISON")
    print("="*70)
    
    # Cell dimensions for mass calculation
    cell_pore_volume = delr[0] * delc[0] * Lz * porosity_1d  # m³ of pore space per cell
    
    # IMPORTANT: Set analytical source location to match MT3D source cell center
    x_centers = np.cumsum(delr) - delr/2
    xc_source = x_centers[0] + 20  # center of first cell + 20m offset to avoid edge effects
    
    print(f"\nSetup:")
    print(f"  Initial mass release: M₀ = {M_instantaneous} kg")
    print(f"  Cell pore volume: {cell_pore_volume:.2f} m³")
    print(f"  Domain: {ncol} cells × {delr[0]:.1f} m = {Lx} m")
    print(f"  Source location: xc = {xc_source:.1f} m (center of first cell)")
    
    # Calculate mass at different times
    print(f"\n{'Time':>8s}  {'MT3D Mass':>12s}  {'Analytical Mass':>16s}  {'MT3D Error':>12s}  {'Analytical Error':>16s}")
    print("-" * 70)
    
    mass_comparison = []
    
    for t_actual in sorted(conc_profiles_mt3d.keys()):
        # MT3D numerical mass
        conc_mt3d = conc_profiles_mt3d[t_actual]
        mass_mt3d = np.sum(conc_mt3d) * cell_pore_volume  # kg
        
        # Analytical mass (integrate over domain) WITH CORRECTED SOURCE LOCATION
        C_analytical = pulse1(m0=M_instantaneous, x=x_centers, t=t_actual, 
                              v=u_1d, n=porosity_1d, al=alpha_L_1d, xc=xc_source)
        mass_analytical = np.sum(C_analytical) * cell_pore_volume  # kg
        
        # Calculate errors
        error_mt3d = abs(mass_mt3d - M_instantaneous) / M_instantaneous * 100
        error_analytical = abs(mass_analytical - M_instantaneous) / M_instantaneous * 100
        
        mass_comparison.append({
            't': t_actual,
            'mt3d': mass_mt3d,
            'analytical': mass_analytical,
            'error_mt3d': error_mt3d,
            'error_analytical': error_analytical
        })
        
        print(f"{t_actual:7.1f}d  {mass_mt3d:11.6f} kg  {mass_analytical:15.6f} kg  {error_mt3d:11.3f}%  {error_analytical:15.3f}%")
    
    # Summary statistics
    print("\n" + "="*70)
    avg_error_mt3d = np.mean([m['error_mt3d'] for m in mass_comparison])
    avg_error_analytical = np.mean([m['error_analytical'] for m in mass_comparison])
    max_error_mt3d = np.max([m['error_mt3d'] for m in mass_comparison])
    max_error_analytical = np.max([m['error_analytical'] for m in mass_comparison])
    
    print(f"SUMMARY:")
    print(f"  MT3D Numerical Model:")
    print(f"    Average mass error: {avg_error_mt3d:.3f}%")
    print(f"    Maximum mass error: {max_error_mt3d:.3f}%")
    
    print(f"\n  Analytical Solution:")
    print(f"    Average mass error: {avg_error_analytical:.3f}%")
    print(f"    Maximum mass error: {max_error_analytical:.3f}%")
    
    # Interpretation
    print(f"\n{'='*70}")
    print("INTERPRETATION:")
    
    if max_error_mt3d < 1.0:
        print(f"  ✓ MT3D: EXCELLENT mass conservation (< 1% error)")
    elif max_error_mt3d < 5.0:
        print(f"  ✓ MT3D: GOOD mass conservation (< 5% error)")
    else:
        print(f"  ⚠ MT3D: POOR mass conservation (> 5% error)")
    
    if max_error_analytical < 1.0:
        print(f"  ✓ Analytical: EXCELLENT mass conservation (< 1% error)")
    elif max_error_analytical < 5.0:
        print(f"  ✓ Analytical: GOOD mass conservation (< 5% error)")
    else:
        print(f"  ⚠ Analytical: POOR mass conservation (> 5% error)")
    
    # Additional notes on analytical solution mass loss
    if max_error_analytical > 0.1:
        print(f"\n  NOTE: Analytical solution may show mass 'loss' because:")
        print(f"    • Integration domain is finite ({Lx} m)")
        print(f"    • Plume spreads beyond domain at later times")
        print(f"    • True analytical solution extends to ±∞")
        print(f"    • This is expected and not a numerical error")
    
    print(f"{'='*70}\n")
    
else:
    print("Skipping mass comparison - MT3D simulation failed")

Why do we have slight discrepancies? The analytical solution assumes instantaneous release of mass at a point, while the numerical solution distributes mass over a finite cell volume that is released after the first time step. This leads to minor differences, especially near the source.

### 4.8 Summary: Numerical Verification Complete

**What we accomplished in Section 4:**

✅ **Created 1D MODFLOW model** with uniform flow (v = 0.4 m/day)  
✅ **Proper grid resolution** - Fine grid with appropriate cell size  
✅ **Steady-state flow** → Passed on to MT3D  
✅ **Instantaneous pulse source** - 10 kg released at t=0  
✅ **MT3D accurately reproduces analytical solution** - Excellent agreement!

**Key takeaway:** MT3D-USGS works correctly and can be trusted for transport predictions when properly configured. This verification step builds confidence before we move to complex applications.

**Next:** Now that we've verified MT3D works, we'll load the Limmat Valley flow model and prepare it for transport modeling using the telescope approach.

---
## 5. Example Application: Limmat Valley Aquifer

Now we will load the existing MODFLOW model of the Limmat Valley aquifer and apply the transport processes using MT3D-USGS to see how a solute behaves in this complex system.

### 5.1 Import Libraries

Import all necessary Python libraries for groundwater modeling, transport simulation, and visualization.

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# FloPy for MODFLOW and MT3D
import flopy

# print current working directory
print("Current working directory: ", os.getcwd())

# Add the support repo to the path
sys.path.append(os.path.abspath('../SUPPORT_REPO/src'))
sys.path.append(os.path.abspath('../SUPPORT_REPO/src/scripts/scripts_exercises'))


# Helper functions from course utilities
from data_utils import download_named_file, get_default_data_folder

print(f"FloPy version: {flopy.__version__}")
print("Libraries imported successfully.")

### 5.2 Load and Run Parent Flow Model

Load the Limmat Valley base model and run steady-state flow simulation. Transport modeling requires a converged flow field to calculate groundwater velocities.

In [ ]:
# Download parent base model
parent_base_flow_model_name = 'baseline_model_flow'

parent_base_flow_model_path = download_named_file(
    'baseline_model', 
    data_type='transport',
)

# Handle zip file extraction if needed
if parent_base_flow_model_path.endswith('.zip'):
    import zipfile
    extract_path = os.path.dirname(parent_base_flow_model_path)
    with zipfile.ZipFile(parent_base_flow_model_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    # Find the .nam file
    parent_base_flow_model_path = os.path.join(extract_path, 'limmat_valley_model_nwt.nam')

print(f'Downloaded parent base model to: {parent_base_flow_model_path}')

In [ ]:
# Define workspace for transport modeling
transport_ws = Path.home() / "applied_groundwater_modelling_data" / "limmat" / "transport"
flow_model_ws = transport_ws 
flow_model_ws.mkdir(parents=True, exist_ok=True)

print(f"Transport workspace: {transport_ws}")
print(f"Flow model workspace: {flow_model_ws}")

In [ ]:
# Load MODFLOW model with FloPy
mf = flopy.modflow.Modflow.load(
    'limmat_valley_model_nwt.nam',
    model_ws=os.path.dirname(parent_base_flow_model_path),
    exe_name='mfnwt',  # MODFLOW-NWT executable
    version='mfnwt',
    check=False
)

print(f"Loaded MODFLOW model: {mf.name}")
print(f"Grid dimensions (nlay, nrow, ncol): {mf.nlay}, {mf.nrow}, {mf.ncol}")
print(f"Steady-state: {mf.dis.steady.array[0]}")

In [ ]:
# Change model workspace to our transport directory and save
mf.change_model_ws(str(flow_model_ws))

# Add LMT package to create flow-transport link file for MT3D
lmt = flopy.modflow.ModflowLmt(mf, output_file_name='mt3d_link.ftl')

# Write input files
mf.write_input()

print(f"Saved flow model to: {flow_model_ws}")
print("Added LMT package for MT3D coupling")

In [ ]:
# Run the flow model (this will create the mt3d_link.ftl file)
success, buff = mf.run_model(silent=False)

if success:
    print("\n✓ Flow model converged successfully")
    print("✓ Created mt3d_link.ftl for MT3D coupling")
else:
    print("\n✗ Flow model failed to converge")
    print("Check output above for errors")

### 5.3 Load and Inspect Flow Results

Load the head file and cell-by-cell flow file to verify the flow solution and extract velocities for transport.

In [ ]:
# Load head file
hds_file = flow_model_ws / f"{mf.name}.hds"
hds = flopy.utils.HeadFile(str(hds_file))

# Get heads for the single steady-state time step
heads = hds.get_data()

print(f"Head array shape: {heads.shape}")
print(f"Head range: {heads.min():.2f} to {heads.max():.2f} m")

In [ ]:
# Visualize head distribution
fig, ax = plt.subplots(figsize=(10, 8))

# Plot heads for layer 0
mapview = flopy.plot.PlotMapView(model=mf, layer=0, ax=ax)
quadmesh = mapview.plot_array(heads, cmap='viridis')
mapview.plot_grid(alpha=0.2)
mapview.plot_ibound()

# Add colorbar
cb = plt.colorbar(quadmesh, ax=ax, shrink=0.35)
cb.set_label('Hydraulic Head (m)', fontsize=12)

ax.set_title('Steady-State Head Distribution (Layer 1)', fontsize=14)
ax.set_xlabel('Easting (m)', fontsize=12)
ax.set_ylabel('Northing (m)', fontsize=12)

ax.set_aspect('equal')

plt.tight_layout()
plt.show()

### 5.4 Define Contaminant source Location

First, we need to choose where the contaminant source is located and map it to the model grid.

In [ ]:
# Get model grid extent first
xoff = mf.modelgrid.xoffset
yoff = mf.modelgrid.yoffset
delr = mf.dis.delr.array
delc = mf.dis.delc.array

# Calculate model domain boundaries
x_min = xoff
x_max = xoff + np.sum(delr)
y_max = yoff  # Top of domain (row 0)
y_min = yoff - np.sum(delc)  # Bottom of domain (row nrow-1)

print(f"Model domain extent:")
print(f"  X (Easting): {x_min:.1f} to {x_max:.1f}")
print(f"  Y (Northing): {y_min:.1f} (bottom) to {y_max:.1f} (top)")

# Alternative approach: Pick a specific row and column, then get its center coordinates
# This avoids intersect() issues entirely
source_row = mf.nrow // 2  # Middle row (24 out of 48)
source_col = int(mf.ncol * 0.80)  # 80% across (113 out of 142)
source_layer = 0  # Top layer (0-indexed in Python)

# Get the cell center coordinates for this row/col
# Method: get corner and add half cell width
x_corner = xoff + np.sum(delr[:source_col])
y_corner = yoff - np.sum(delc[:source_row])
source_easting = x_corner + delr[source_col] / 2
source_northing = y_corner - delc[source_row] / 2

print(f"\nChosen grid cell: Row {source_row}, Col {source_col}, Layer {source_layer}")
print(f"Cell center coordinates:")
print(f"  Easting: {source_easting:.1f}")
print(f"  Northing: {source_northing:.1f}")

# Verify using intersect (for confirmation only, we already have row/col)
try:
    row_check, col_check = mf.modelgrid.intersect(source_easting, source_northing)
    if row_check == source_row and col_check == source_col:
        print(f"\n✓ Verified: intersect confirms Row {row_check}, Col {col_check}")
    else:
        print(f"\n⚠ Mismatch: intersect gives Row {row_check}, Col {col_check}")
except:
    print(f"\n✓ Using direct grid indices (intersect not needed)")
    
# Set final values
row = source_row
col = source_col

# Check that source location variables are defined
if 'source_easting' not in locals() or 'row' not in locals():
    print("⚠ Please run cell 3.2 first to define source location!")
else:
    fig, ax = plt.subplots(figsize=(12, 10))

    # Plot model grid and heads
    mapview = flopy.plot.PlotMapView(model=mf, layer=source_layer, ax=ax)
    quadmesh = mapview.plot_array(heads, cmap='Blues', alpha=0.5)
    mapview.plot_grid(alpha=0.2, linewidth=0.5)
    mapview.plot_ibound()

    # Add colorbar for heads
    cb = plt.colorbar(quadmesh, ax=ax, shrink=0.3)
    cb.set_label('Hydraulic Head (m)', fontsize=11)

    # Mark source location using grid cell center
    # Get cell vertices to find center
    vertices = mf.modelgrid.get_cell_vertices(row, col)
    cell_x = np.mean([v[0] for v in vertices])
    cell_y = np.mean([v[1] for v in vertices])
    
    # Plot source location
    ax.plot(cell_x, cell_y, 'r*', markersize=12, 
            label=f'Source (Row {row}, Col {col})', zorder=10, markeredgecolor='black', markeredgewidth=1)
    
    ax.set_title('Contaminant Source Location on Model Grid', fontsize=14, pad=20)
    ax.set_xlabel('Easting (m)', fontsize=12)
    ax.set_ylabel('Northing (m)', fontsize=12)
    ax.legend(fontsize=11, loc='upper left', framealpha=0.9)
    ax.set_aspect('equal')

    plt.tight_layout()
    plt.show()

    print(f"\n✓ Source marked at grid cell center:")
    print(f"  Grid indices: Row {row}, Col {col}, Layer {source_layer}")
    print(f"  Cell center: E={cell_x:.1f}, N={cell_y:.1f}")
    print(f"  Look for RED STAR with black edge and red circle")

There are multiple options to define a contaminant source in MT3D-USGS. Below, we discuss two common approaches for defining a continuous source.

**Continuous Source Definition:**

For continuous releases (industrial discharge, wastewater outfall, ongoing contamination), we define a constant concentration or constant mass loading rate at the source location.

Two Approaches for Continuous Sources:

**Option 1: Constant Concentration Source (icbund = -1)**

- Use when you know the concentration at the source (e.g., effluent discharge at fixed concentration)
- Set icbund[source_cell] = -1 to maintain constant concentration
- MT3D will automatically inject/remove mass to maintain this concentration
- Mass balance file shows cumulative mass added

**Option 2: Constant Mass Flux via SSM Package**

- Use when you know the mass loading rate (e.g., 10 kg/day)
- Use Source-Sink Mixing (SSM) package to inject constant mass
- Can link to MODFLOW stresses (wells, rivers) or inject directly

**Implementation for Option 1 (Constant Concentration):**

- Define source concentration: C_source (kg/m³ or mg/L)
- Set boundary condition: icbund[source_cell] = -1
- Set initial concentration: sconc[source_cell] = C_source
- Transport occurs: concentration maintained at source, plume spreads downgradient
Eventually reaches steady-state (if domain is large enough)

**Implementation for Option 2 (Constant Mass Flux):**

- Define mass loading rate: M_rate (kg/day)
- Keep icbund[source_cell] = 1 (active cell)
- Set initial concentration: sconc[source_cell] = 0 (or background)
- Use SSM package to inject mass at constant rate
- Concentration at source varies based on mass balance

This differs from an instantaneous source which releases all mass at t=0 and then decays away.

**Physical meaning (Option 1):**

- t=0: Source concentration = C_source, rest of domain = 0
- t>0: Plume grows, concentration maintained at source
- t→∞: Steady-state plume (if boundary conditions allow)
- Total mass in system increases over time until steady-state

For this notebook, we'll demonstrate Option 1 (constant concentration) as it's simpler and more commonly used for industrial discharge scenarios.

In [ ]:
# Create MT3D model for 2D Limmat Valley transport
# Using the source location from Section 3
# CONTINUOUS SOURCE CONFIGURATION

# Transport parameters
porosity = 0.25  # Effective porosity (dimensionless)
alpha_L = 10.0  # Longitudinal dispersivity (m)
alpha_TH = 1.0  # Transverse horizontal dispersivity (m)  
alpha_TV = 0.1  # Transverse vertical dispersivity (m)

# Check Peclet number for 2D model
dx_2d = mf.dis.delr.array[0]  # Grid spacing (assumes uniform)
Pe_2d = dx_2d / alpha_L
print(f"2D Model Grid Resolution Check:")
print(f"  Δx = {dx_2d:.1f} m")
print(f"  αL = {alpha_L} m")
print(f"  Pe = Δx/αL = {Pe_2d:.2f}")
if Pe_2d <= 2:
    print(f"  ✓ Pe ≤ 2: Good resolution!")
else:
    print(f"  ❌ Pe = {Pe_2d:.1f} > 2: Grid too coarse - expect numerical artifacts!")
    print(f"  (Compare to Section 4 where Pe = 0.5 gave excellent results)")

# Simulation time
sim_time = 3650  # 10 years in days
nper = 1
perlen = [sim_time]

# Time stepping - use 4-day steps 
dt = 4.0  # days
nstp = [int(np.ceil(sim_time / dt))]
print(f"\nTime stepping:")
print(f"  Δt = {dt} days")
print(f"  Number of steps: {nstp[0]}")
print(f"  Total simulation time: {sim_time} days ({sim_time/365:.1f} years)")

# Create MT3D model
mt = flopy.mt3d.Mt3dms(
    modelname='mt_' + mf.name,
    model_ws=str(flow_model_ws),
    exe_name='mt3dusgs',
    version="mt3d-usgs",
    modflowmodel=mf
)

print(f"\n✓ Created MT3D model: {mt.name}")
print(f"✓ Linked to MODFLOW model: {mf.name}")
print(f"\n{'='*60}")
print(f"SOURCE CONFIGURATION: CONTINUOUS RELEASE")
print(f"{'='*60}")

In [ ]:
# BTN Package - Basic Transport
# CONTINUOUS SOURCE: Constant concentration maintained at source cell

# icbund: 1 = active transport, 0 = inactive, -1 = CONSTANT CONCENTRATION
icbund = np.ones((mf.nlay, mf.nrow, mf.ncol), dtype=np.int32)

# Set source cell to CONSTANT CONCENTRATION (icbund = -1)
# This maintains the concentration at C_source for the entire simulation
icbund[source_layer, row, col] = -1

# Match inactive flow cells (where ibound <= 0)
ibound = mf.bas6.ibound.array
icbund[ibound <= 0] = 0

print(f"Transport boundary conditions (icbund):")
print(f"  Active transport cells: {np.sum(icbund == 1)}")
print(f"  Constant concentration cells: {np.sum(icbund == -1)} ← CONTINUOUS SOURCE")
print(f"  Inactive transport cells: {np.sum(icbund == 0)}")

# Initial concentration array
sconc = np.zeros((mf.nlay, mf.nrow, mf.ncol), dtype=np.float32)

# Define SOURCE CONCENTRATION (this will be maintained throughout simulation)
# Option A: Define in mg/L and convert to kg/m³
C_source_mg_L = 100.0  # mg/L - typical for industrial effluent
C_source = C_source_mg_L * 1e-3  # Convert mg/L to kg/m³

# Option B: Or define directly in kg/m³
# C_source = 0.1  # kg/m³ (equivalent to 100 mg/L)

# Set initial concentration at source cell
sconc[source_layer, row, col] = C_source

print(f"\nContinuous source concentration:")
print(f"  C_source = {C_source:.6f} kg/m³ = {C_source*1e3:.2f} mg/L")
print(f"  Location: Layer {source_layer}, Row {row}, Col {col}")
print(f"  This concentration will be MAINTAINED throughout the simulation")

# Calculate cell volume for reference (not needed for constant concentration, but useful for diagnostics)
cell_volume = delr[col] * delc[row] * (mf.dis.top.array[row, col] - mf.dis.botm.array[source_layer, row, col])
pore_volume = cell_volume * porosity
print(f"\nCell geometry:")
print(f"  Cell volume: {cell_volume:.2f} m³")
print(f"  Pore volume: {pore_volume:.2f} m³")

# Porosity array
prsity = np.ones((mf.nlay, mf.nrow, mf.ncol), dtype=np.float32) * porosity

# Output times - save at regular intervals
nprs = 20
timprs = np.linspace(sim_time / nprs, sim_time, nprs)

btn = flopy.mt3d.Mt3dBtn(
    mt,
    ncomp=1,  # Number of species (1 for conservative tracer)
    mcomp=1,  # Number of mobile species
    munit='KG',  # Mass units (kg)
    tunit='D',  # Time units (days)
    lunit='M',  # Length units (meters)
    prsity=prsity,
    icbund=icbund,
    sconc=sconc,
    nper=nper,
    perlen=perlen,
    nstp=nstp,
    tsmult=[1.0],  # Time step multiplier (1.0 = constant)
    timprs=timprs,  # Times at which to save concentration
    dt0=dt  # Initial time step size
)

print(f"\n✓ BTN package created")
print(f"  Source type: CONTINUOUS (icbund = -1)")
print(f"  Source concentration: {C_source:.6f} kg/m³ ({C_source*1e3:.2f} mg/L)")
print(f"  Initial concentration at [{source_layer},{row},{col}]: {sconc[source_layer, row, col]:.6f} kg/m³")

# ADV Package - Advection
# mixelm = -1: TVD scheme (Total Variation Diminishing, recommended for sharp fronts)
adv = flopy.mt3d.Mt3dAdv(mt, mixelm=-1)
print(f"✓ ADV package created (TVD scheme)")

# DSP Package - Dispersion
dsp = flopy.mt3d.Mt3dDsp(
    mt,
    al=alpha_L,  # Longitudinal dispersivity
    trpt=alpha_TH / alpha_L,  # Ratio of transverse to longitudinal (horizontal)
    trpv=alpha_TV / alpha_L,  # Ratio of transverse to longitudinal (vertical)
    dmcoef=1e-9  # Molecular diffusion coefficient (m²/day, usually negligible)
)
print(f"✓ DSP package created")
print(f"  αL = {alpha_L} m, αTH = {alpha_TH} m, αTV = {alpha_TV} m")

# SSM Package - Source-Sink Mixing
# For constant concentration source (icbund=-1), SSM is empty
# MT3D automatically handles mass injection to maintain concentration
ssm = flopy.mt3d.Mt3dSsm(mt)
print(f"✓ SSM package created (empty - using icbund=-1 for continuous source)")

# GCG Package - Generalized Conjugate Gradient Solver
gcg = flopy.mt3d.Mt3dGcg(
    mt,
    mxiter=100,  # Maximum outer iterations
    iter1=50,  # Maximum inner iterations
    isolve=3,  # Solver type (3 = Jacobi preconditioned CG)
    cclose=1e-6  # Convergence criterion
)
print(f"✓ GCG solver package created")
print(f"\n{'='*60}")
print(f"MT3D MODEL SETUP COMPLETE - CONTINUOUS SOURCE")
print(f"{'='*60}")
print(f"Ready to run transport simulation!")
print(f"\nExpected behavior:")
print(f"  • Concentration at source remains constant at {C_source*1e3:.2f} for the first time step mg/L")
print(f"  • Plume grows downgradient over time")
print(f"  • Total mass in system increases until steady-state")
print(f"  • Check mass balance file (.MAS) for mass injection rate")

### 5.5 Set Up MT3D Packages
Configure all MT3D packages: BTN (basic transport), ADV (advection), DSP (dispersion), SSM (sources), and GCG (solver).

**Optional: RCT Package for Reactive Transport**
For conservative tracers (no sorption or decay), we don't need RCT. For reactive transport, you would add:

**Example: with sorption and decay**
rct = flopy.mt3d.Mt3dRct(
    mt,
    isothm=1,  # Sorption type (1=linear, 2=Freundlich, 3=Langmuir)
    sp1=0.5,   # Kd distribution coefficient (mL/g)
    rc1=0.001  # First-order decay constant (1/day)
)
For this demonstration, we're using a conservative tracer (no RCT package).

### 5.6 Write and Run MT3D Model
Note: FloPy sometimes incorrectly reports MT3D runs as "failed" even when they complete successfully. We check for the output file (.ucn) to verify actual success.

In [ ]:
# Clean up ALL old output files first
print("Cleaning up old output files...")
for file_pattern in ["MT3D*.UCN", "MT3D*.MAS", "MT3D*.OBS", "*.ucn"]:
    for old_file in flow_model_ws.glob(file_pattern):
        old_file.unlink()
        print(f"  ✓ Deleted: {old_file.name}")

# Write MT3D input files
mt.write_input()
print(f"\n✓ MT3D input files written to: {mt.model_ws}")

# Run MT3D simulation
print("\nRunning MT3D-USGS transport simulation...")
print(f"(This may take a few minutes with {nstp[0]} time steps)")
success, buff = mt.run_model(silent=False)  # Changed to see error messages

# Check for output file - MT3D uses default name MT3D001.UCN
ucn_file_default = flow_model_ws / "MT3D001.UCN"

if ucn_file_default.exists() and ucn_file_default.stat().st_size > 0:
    print("\n✓ Transport simulation completed successfully")
    print(f"✓ Concentration file created: {ucn_file_default.name}")
    
elif ucn_file_default.exists():
    print("\n✗ Transport simulation failed - UCN file is empty")
    print("Check MT3D output above for errors")
    print("\nCommon issues:")
    print("  - Time step too small (try increasing Δt)")
    print("  - Memory issues with large number of time steps")
    print("  - MT3D convergence failure")
else:
    print("\n✗ Transport simulation failed - no concentration output")
    print("Check output above for errors")

### 5.7 Load Concentration Results & Diagnostic Analysis of Concentrations
Load the concentration file (.UCN) and extract results at different times and check concentration values for physical reasonableness and numerical artifacts.

In [ ]:
# Load concentration file - MT3D uses default name MT3D001.UCN
ucn_file = flow_model_ws / "MT3D001.UCN"
if not ucn_file.exists():
    ucn_file = flow_model_ws / f"{mt.name}.ucn"  # Try model-specific name

ucn = flopy.utils.UcnFile(str(ucn_file))

# Get available times
times = ucn.get_times()
print(f"Concentration saved at {len(times)} times:")
print(f"  Times (days): {[f'{t:.0f}' for t in times[:5]]}...")

# Load concentration at FIRST and FINAL time
conc_first = ucn.get_data(totim=times[0])
conc_final = ucn.get_data(totim=times[-1])

# CRITICAL: Mask inactive cells to ignore uninitialized memory values
# Only analyze cells where transport is active (icbund != 0)
active_mask = icbund != 0
conc_first_active = np.where(active_mask, conc_first, np.nan)
conc_final_active = np.where(active_mask, conc_final, np.nan)

print(f"\n{'='*60}")
print(f"DIAGNOSTIC CHECK - CONTINUOUS SOURCE")
print(f"Concentration in kg/m³ (ACTIVE CELLS ONLY)")
print(f"{'='*60}")

print(f"\nFirst timestep (t={times[0]:.1f} days):")
print(f"  Min: {np.nanmin(conc_first_active):.6f} kg/m³")
print(f"  Max: {np.nanmax(conc_first_active):.6f} kg/m³")
print(f"  At source [{source_layer},{row},{col}]: {conc_first[source_layer, row, col]:.6f} kg/m³")
print(f"  Source concentration (fixed): {C_source:.6f} kg/m³")

print(f"\nFinal timestep (t={times[-1]:.1f} days = {times[-1]/365:.1f} years):")
print(f"  Min: {np.nanmin(conc_final_active):.6f} kg/m³")
print(f"  Max: {np.nanmax(conc_final_active):.6f} kg/m³")
print(f"  Mean (active cells): {np.nanmean(conc_final_active):.6f} kg/m³")
print(f"  At source [{source_layer},{row},{col}]: {conc_final[source_layer, row, col]:.6f} kg/m³")

# Verify source concentration is maintained
source_conc_final = conc_final[source_layer, row, col]
if abs(source_conc_final - C_source) / C_source < 0.01:  # Within 1%
    print(f"\n✓ Source concentration maintained correctly!")
    print(f"  Deviation: {abs(source_conc_final - C_source)/C_source*100:.2f}%")
else:
    print(f"\n⚠ WARNING: Source concentration not maintained!")
    print(f"  Expected: {C_source:.6f} kg/m³")
    print(f"  Actual: {source_conc_final:.6f} kg/m³")
    print(f"  Deviation: {abs(source_conc_final - C_source)/C_source*100:.2f}%")

# Check for physically impossible values (in ACTIVE cells only)
if np.nanmax(conc_final_active) > C_source * 1.01:  # Allow 1% tolerance
    print(f"\n⚠ WARNING: Maximum concentration ({np.nanmax(conc_final_active):.6f}) exceeds source!")
    print(f"  This indicates numerical instability or configuration error.")
    print(f"  Ratio: {np.nanmax(conc_final_active) / C_source:.2f}x source concentration")

    # Find where max occurs IN ACTIVE CELLS
    max_loc = np.unravel_index(np.nanargmax(conc_final_active), conc_final_active.shape)
    print(f"  Max occurs at: Layer {max_loc[0]}, Row {max_loc[1]}, Col {max_loc[2]}")
    print(f"  icbund at max location: {icbund[max_loc]}")

    # Check how many ACTIVE cells exceed source
    n_exceed = np.sum(conc_final_active > C_source)
    print(f"  Active cells exceeding source concentration: {n_exceed}")
else:
    print(f"\n✓ Maximum concentration is physically reasonable")

# Convert to mg/L for display (active cells only)
conc_final_mg_L = conc_final_active * 1e3
print(f"\n{'='*60}")
print(f"Display units (mg/L) - ACTIVE CELLS ONLY:")
print(f"  Source concentration: {C_source*1e3:.2f} mg/L (maintained)")
print(f"  Max concentration: {np.nanmax(conc_final_mg_L):.2f} mg/L")
print(f"  Mean concentration: {np.nanmean(conc_final_mg_L):.4f} mg/L")
print(f"  Cells with C > 1 mg/L: {np.sum(conc_final_mg_L > 1.0)}")
print(f"  Cells with C > 10 mg/L: {np.sum(conc_final_mg_L > 10.0)}")
print(f"{'='*60}")
print(f"\n✓ Solution: Masked {np.sum(~active_mask)} inactive cells to ignore uninitialized values")

# Load and display mass balance information
mas_file = flow_model_ws / "MT3D001.MAS"
if mas_file.exists():
    print(f"\n{'='*60}")
    print(f"MASS BALANCE (from {mas_file.name}):")
    print(f"{'='*60}")
    print(f"Check this file for:")
    print(f"  • Total mass in system over time")
    print(f"  • Mass injection rate at source (from constant concentration BC)")
    print(f"  • Mass leaving domain through boundaries")
    print(f"  • Mass balance error (should be < 1%)")
    print(f"\nTo view: !cat '{mas_file}' | tail -50")

We have oscillations and non-physical behavior in the concentration profiles due to numerical dispersion.

### 5.8 Visualize Concentration Over Time
Plot concentration maps at multiple times to see plume migration.

In [ ]:
# Visualize concentration at multiple times - CONTINUOUS SOURCE
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

# Select 4 times to plot - adapt to available times
if len(times) >= 4:
    # Use evenly spaced times if enough are available
    indices = [0, len(times)//3, 2*len(times)//3, -1]
    plot_times = [times[i] for i in indices]
else:
    # Use all available times
    plot_times = times
    # Hide extra subplots if fewer than 4 times
    for j in range(len(times), 4):
        axes[j].set_visible(False)

# Track concentration at source over time for diagnostics
source_conc_over_time = []

for i, t in enumerate(plot_times):
    ax = axes[i]
    conc = ucn.get_data(totim=t)
    
    # Track source concentration
    source_conc_over_time.append(conc[source_layer, row, col])
    
    # Apply active cell mask to exclude inactive cells with garbage values
    conc_masked = np.where(active_mask, conc, np.nan)
    
    # Convert to mg/L for display (multiply by 1e3)
    conc_mg_L = conc_masked * 1e3
    
    # Plot concentration
    mapview = flopy.plot.PlotMapView(model=mf, layer=0, ax=ax)
    quadmesh = mapview.plot_array(conc_mg_L, cmap='YlOrRd', vmin=0, vmax=C_source*1e3)
    mapview.plot_grid(alpha=0.2)
    
    # Mark source location
    vertices = mf.modelgrid.get_cell_vertices(row, col)
    cell_x = np.mean([v[0] for v in vertices])
    cell_y = np.mean([v[1] for v in vertices])
    ax.plot(cell_x, cell_y, 'r*', markersize=15, zorder=10, 
            markeredgecolor='black', markeredgewidth=1.5, label='Continuous Source')
    
    # Add colorbar
    cb = plt.colorbar(quadmesh, ax=ax, shrink=0.34)
    cb.set_label('Concentration (mg/L)', fontsize=10)
    
    # Add title with source concentration info
    source_c_current = conc[source_layer, row, col] * 1e3
    ax.set_title(f'Time = {t:.0f} days ({t/365:.1f} years)\nSource: {source_c_current:.1f} mg/L (constant)', 
                 fontsize=11)
    ax.set_xlabel('Easting (m)', fontsize=10)
    ax.set_ylabel('Northing (m)', fontsize=10)
    ax.set_aspect('equal')
    ax.legend(loc='upper right', fontsize=8)

plt.tight_layout()
plt.show()

print(f"\n{'='*60}")
print(f"CONTINUOUS SOURCE PLUME DEVELOPMENT")
print(f"{'='*60}")
print(f"Visualized {len(plot_times)} time steps from {times[0]:.0f} to {times[-1]:.0f} days")
print(f"  Source type: CONTINUOUS (concentration maintained)")
print(f"  Source concentration: {C_source:.6f} kg/m³ ({C_source*1e3:.2f} mg/L)")
print(f"  Color scale: 0 to {C_source*1e3:.2f} mg/L (source concentration)")
print(f"\nSource concentration over time:")
for i, (t, c) in enumerate(zip(plot_times, source_conc_over_time)):
    print(f"  t = {t:5.0f} days: C = {c*1e3:.2f} mg/L")
print(f"\n✓ Red stars mark continuous source location at Row {row}, Col {col}")
print(f"✓ Inactive cells (with garbage values) are masked and shown as white/no data")
print(f"\nObservations:")
print(f"  • Plume grows over time from the continuous source")
print(f"  • Source concentration remains constant at {C_source*1e3:.2f} mg/L")
print(f"  • Plume extends further downgradient with time")
print(f"  • Eventually will reach steady-state (if simulation runs long enough)")

## 6. Alternative Approaches and Method Selection
### 6.1 Learning Objectives
After completing this section, you will be able to:

- Understand when to use different transport modeling approaches
- Select appropriate methods based on problem requirements
- Recognize the advantages and limitations of each approach
- 
### 6.2 The Modeling Spectrum: From Simple to Complex
Transport modeling offers a range of approaches, from simple analytical solutions to complex numerical simulations. 

**The key is choosing the right tool for the problem at hand.**

### 6.3 Method Categories and Their Characteristics

#### 6.3.1 Analytical Solutions
**When to use:**

- Homogeneous, simple geology
- Constant flow field
- Simple boundary conditions
- Quick screening or verification

**Limitations:**
- Cannot handle complex heterogeneity
- Limited to idealized conditions
- No irregular boundaries

**Example:** The 1D pulse source solution we verified in Section 3 and 4.

#### 6.3.2 Numerical Models (MT3DMS, MODFLOW 6 GWT)
**When to use:**
- Complex heterogeneous geology
- Multiple sources and sinks
- Time-varying conditions
- Need for detailed concentration predictions

**Limitations:**
- Requires careful grid design
- More computationally intensive
- Need verification against simpler cases

**Example:** The telescope submodel approach we'll use for the case study.

#### 6.3.3 Particle Tracking (MODPATH)
**When to use:**
- Advection-dominated transport
- Capture zone delineation
- Travel time analysis
- Preliminary site assessments

**Limitations:**
- Does not model dispersion or reactions
- Qualitative for concentration predictions
- Assumes steady-state flow

**Example:** Wellhead protection zone delineation.

#### 6.3.4 Hybrid Approaches
**Combining methods can leverage the strengths of each:**
- Use particle tracking for preliminary assessment, then MT3D for detailed modeling
- Verify numerical models against analytical solutions where possible
- Use telescope refinement to focus computational effort where needed

### 6.4 Decision Framework for Method Selection
**Key questions to guide method selection:**

1. **What level of detail do you need?**   
- Screening → Analytical or particle tracking   
- Detailed predictions → Numerical (MT3D)

2. **How complex is your site?**   
- Simple/homogeneous → Analytical possible   
- Heterogeneous/complex → Numerical required3. 
 
3. **What processes are important?**   
- Advection only on steady-state flow field → Particle tracking   
- Advection + dispersion → Analytical or MT3D   
- Reactions/decay → MT3D with reaction packages4. 
 
4. **What are your computational constraints?**   
- Limited resources → Start simple, use local refinement   
- More resources → Full 3D transport if justified

### 6.5 Professional Practice: Method Selection in Real Projects
In practice, projects typically follow a tiered approach:
**Phase 1 - Screening:** Analytical solutions or particle tracking for preliminary assessment

**Phase 2 - Calibration:** Numerical models with field data integration

**Phase 3 - Prediction:** Refined numerical models for detailed predictions

**Key principle:** Start simple, add complexity only when needed and justified.

### 6.6 Common Pitfalls in Method Selection
1. **Overcomplicating early:** Using complex numerical models when simpler approaches suffice
2. **Underestimating requirements:** Using particle tracking when dispersion is important
3. **Ignoring verification:** Not checking numerical models against analytical solutions
4. **Wrong grid resolution:** Using grid cells that are too coarse for the dispersivity

### 6.7 Key Takeaways for Method Selection
- **Match method to problem complexity** - don't over- or under-engineer
- **Verify numerical models** against analytical solutions when possible
- **Use local grid refinement** (telescope approach) to focus computational effort
- **Start simple** and add complexity incrementally
- **Consider computational efficiency** 
- fine grids everywhere are rarely needed

### 6.8 Looking Ahead to Section 7
The telescope approach (Section 7) is our solution to the grid resolution challenge. It allows us to:

- Use appropriate grid resolution around wells and sources
- Keep the regional model computationally manageable
- Maintain connection between local and regional scales
  
This is the approach you'll implement for your transport case studies.

## 7. Telescope Approach: Solving the Grid Resolution Problem

### 7.1 Recap: The Problem We Need to Solve

**What we learned from verification (Section 4):**
- With fine grid (Δx = 5m, αL = 10m) → Pe = 0.5 → Excellent agreement with analytical solution ✓
- This established our target: **Pe ≤ 2 for accurate transport modeling**

**What we face in practice (Section 5):**
- Limmat Valley parent model has Δx = 50m grid cells
- For transport with αL = 10m → Pe = 50/10 = 5.0
- **Problem**: Pe = 5.0 > 2 → Grid too coarse for accurate contaminant transport
- **Requirement**: To achieve Pe ≤ 2, we need Δx ≤ 20m (ideally 5-10m for Pe ≤ 1)

**Why we can't just refine the entire model:**
- Refining the entire Limmat Valley to 10m cells would create ~40,000 cells (was 1,000)
- Simulation time would increase by 40× or more
- Most of the domain doesn't need fine resolution
- We only need detail around the contaminant source and wells

**The solution: Telescope (local grid refinement)**
- Create a small refined "child" model extracted from the parent
- Use fine grid (5-10m) only where needed → Pe ≤ 1
- Inherit boundary conditions from the parent model
- Keep most of the domain at coarse resolution

### 7.2 What is the Telescope Approach?

The telescope approach creates a nested "child" model with:
1. **Fine grid** in the area of interest (e.g., 10m cells around wells)
2. **Inherits boundary conditions** from the coarse parent model
3. **Much smaller domain** - only the refined area

**Analogy:** Like using a telescope to zoom in on one part of the sky while the wide-angle view provides context.

### 7.3 When is Telescope Approach Appropriate?

**Use telescope refinement when:**
- You need fine resolution in a localized area (wells, sources)
- Regional model is too coarse for local processes
- Refining the entire model would be too large
- Regional flow field is relatively smooth (not highly variable)

**Don't use telescope if:**
- You need fine resolution everywhere
- Strong heterogeneity at regional scale affects local flow
- Multiple widely-separated areas need refinement (consider multiple submodels)

### 7.4 Telescope Approach Workflow

**Step 1:** Run parent (regional) flow model at coarse resolution

**Step 2:** Extract boundary conditions along submodel perimeter from parent

**Step 3:** Create child submodel with:
- Fine grid around area of interest
- Specified head or flux BC from parent
- Same hydraulic properties (interpolated to fine grid)

**Step 4:** Run child flow model, verify it matches parent at boundaries

**Step 5:** Add transport to child model with fine grid

**Step 6:** Analyze transport results with confidence in grid resolution

### 7.5 Conceptual Example: Our Limmat Valley Case

**Parent model:**
- Domain: 7 km × 7 km
- Grid: 50m cells → 142 × 142 × 3 layers = ~60,000 cells
- Purpose: Regional flow field

**Child submodel (around source/wells):**
- Domain: 1 km × 1 km
- Grid: 10m cells → 100 × 100 × 3 layers = 30,000 cells
- Purpose: Transport modeling with Pe = 1.0

**Result:** Fine resolution where needed, manageable model size.

### 7.6 Key Considerations for Telescope Modeling

#### 7.6.1 Boundary Condition Selection

**Options:**
- **Specified head (CHD)**: Use parent head along submodel boundary
- **Specified flux (WEL)**: Use parent flux across boundaries
- **General head boundary (GHB)**: Allows some flow interaction

**Best practice:** Specified head is simplest and most stable.

#### 7.6.2 Buffer Zone Sizing

The submodel should extend beyond your area of interest:
- **Minimum:** Plume extent + 200m buffer
- **Better:** Capture zone of wells + 300-500m buffer
- **Principle:** Don't let the plume reach the boundary

#### 7.6.3 Grid Resolution Selection

Choose child grid size based on:
- **Dispersivity:** Δx ≤ 2 × αL (keep Pe ≤ 2)
- **Source/well size:** Multiple cells per well
- **Computational limits:** Balance accuracy vs runtime

### 7.7 Advantages and Limitations

**Advantages:**  
✅ Fine resolution where needed  
✅ Manageable model size  
✅ Leverages regional model results  
✅ Standard practice in consulting  

**Limitations:**  
⚠️ Assumes parent flow field is accurate  
⚠️ Boundary conditions are fixed (no feedback to parent)  
⚠️ Requires careful boundary placement  
⚠️ Property interpolation from parent to child  

### 7.8 Verification and Quality Checks

After creating the submodel, verify:
1. **Head continuity:** Child heads match parent at boundaries
2. **Flow continuity:** Flow directions consistent between models
3. **Mass balance:** Check submodel water balance
4. **Grid resolution:** Verify Pe ≤ 2 in transport areas

### 7.9 Looking Ahead: Your Transport Case Study

For your case study, you will:
1. Use the parent Limmat Valley flow model (provided)
2. Create a telescope submodel around your assigned wells
3. Implement transport with appropriate grid resolution
4. Analyze contaminant migration and well capture

**This is a professional workflow** used in consulting projects worldwide.

### 7.10 Summary: Why Telescope Approach Matters

The telescope approach is **essential** for practical transport modeling:
- Solves the grid resolution challenge
- Keeps models computationally feasible
- Focuses detail where it matters
- Industry-standard practice

**Next:** We'll implement a complete telescope submodel with transport in the student case study notebooks.

## 8. Summary and Preparation for Transport Case Study


### 8.1 What We've Learned

Through this comprehensive notebook, we've built a complete understanding of contaminant transport modeling:

#### **1. Fundamental Transport Processes**
- **Advection**: Movement with groundwater flow (dominant at field scale)
- **Dispersion**: Mechanical mixing + molecular diffusion (αL, αT, αV)
- **Sorption**: Contaminant-solid partitioning (Kd, retardation factor R)
- **Decay**: Biodegradation or chemical transformation (λ)

#### **2. Analytical Solutions (Section 3)**
- 1D pulse source solution
- Verification baseline for numerical models
- Understand fundamental behavior

#### **3. Numerical Verification (Section 4)**
- MT3D-USGS reproduces analytical solution
- Build confidence in numerical code
- Importance of proper grid resolution

#### **4. Regional Flow Model (Section 5)**
- Limmat Valley steady-state flow
- Foundation for transport modeling
- Connection to flow case study

#### **5. Method Selection (Section 6)**
- When to use analytical vs numerical methods
- Trade-offs and limitations
- Professional practice guidance

#### **6. Telescope Approach (Section 7)**
- **Critical skill**: Local grid refinement
- Fine resolution where needed
- Industry-standard workflow
- Parent model → child submodel → transport

#### **7. Practical Implementation**
- Create submodel with appropriate boundaries
- Set up transport with proper grid resolution
- Verify and analyze results